# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Your code here
homepage_df = pd.read_csv('homepage_actions.csv')
homepage_df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [3]:
homepage_df.head(20)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [4]:
homepage_df.index

RangeIndex(start=0, stop=8188, step=1)

In [5]:
homepage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [6]:
homepage_df.nunique()

timestamp    8188
id           6328
group           2
action          2
dtype: int64

In [7]:
homepage_df.id.value_counts()

750053    2
389447    2
283040    2
918794    2
208714    2
         ..
467818    1
629924    1
820336    1
424916    1
461199    1
Name: id, Length: 6328, dtype: int64

In [8]:
homepage_df.groupby('id')['action'].value_counts()

id      action
182988  view      1
182994  click     1
        view      1
183089  view      1
183136  view      1
                 ..
937108  view      1
937139  click     1
        view      1
937217  click     1
        view      1
Name: action, Length: 8188, dtype: int64

In [9]:
test_group_a = homepage_df.groupby('id')['action'].agg(list)
test_group_a

id
182988           [view]
182994    [view, click]
183089           [view]
183136           [view]
183141    [view, click]
              ...      
937003           [view]
937073           [view]
937108           [view]
937139    [view, click]
937217    [view, click]
Name: action, Length: 6328, dtype: object

In [10]:
test_group_a.value_counts()

[view]           4468
[view, click]    1860
Name: action, dtype: int64

In [11]:
df_3 = homepage_df.groupby(['id','group'])['action'].agg(list)
df_3

id      group     
182988  experiment           [view]
182994  control       [view, click]
183089  control              [view]
183136  experiment           [view]
183141  experiment    [view, click]
                          ...      
937003  control              [view]
937073  control              [view]
937108  control              [view]
937139  experiment    [view, click]
937217  control       [view, click]
Name: action, Length: 6328, dtype: object

In [12]:
df_4 = homepage_df.groupby(['id','group'])['action'].agg(list).reset_index(name='user_actions')
df_4

,id,group,user_actions
0,182988,experiment,[view]
1,182994,control,"[view, click]"
2,183089,control,[view]
3,183136,experiment,[view]
4,183141,experiment,"[view, click]"
...,...,...,...
6323,937003,control,[view]
6324,937073,control,[view]
6325,937108,control,[view]
6326,937139,experiment,"[view, click]"


In [13]:
df_4['user_actions']

0              [view]
1       [view, click]
2              [view]
3              [view]
4       [view, click]
            ...      
6323           [view]
6324           [view]
6325           [view]
6326    [view, click]
6327    [view, click]
Name: user_actions, Length: 6328, dtype: object

In [14]:
df_4[['viewed', 'clicked']] = pd.DataFrame(df_4.user_actions.tolist(), index= df_4.index)
df_4

,id,group,user_actions,viewed,clicked
0,182988,experiment,[view],view,None
1,182994,control,"[view, click]",view,click
2,183089,control,[view],view,None
3,183136,experiment,[view],view,None
4,183141,experiment,"[view, click]",view,click
...,...,...,...,...,...
6323,937003,control,[view],view,None
6324,937073,control,[view],view,None
6325,937108,control,[view],view,None
6326,937139,experiment,"[view, click]",view,click


In [15]:
df_4 = df_4.drop(columns='user_actions')
df_4

,id,group,viewed,clicked
0,182988,experiment,view,None
1,182994,control,view,click
2,183089,control,view,None
3,183136,experiment,view,None
4,183141,experiment,view,click
...,...,...,...,...
6323,937003,control,view,None
6324,937073,control,view,None
6325,937108,control,view,None
6326,937139,experiment,view,click


In [16]:
df_4['viewed'] = df_4['viewed'].map({'view': 1, 'None': 0}).astype('Int64')
df_4['clicked'] = df_4['clicked'].map({'click': 1, 'None': 0}).astype('Int64').fillna(0)
df_4

,id,group,viewed,clicked
0,182988,experiment,1,0
1,182994,control,1,1
2,183089,control,1,0
3,183136,experiment,1,0
4,183141,experiment,1,1
...,...,...,...,...
6323,937003,control,1,0
6324,937073,control,1,0
6325,937108,control,1,0
6326,937139,experiment,1,1


In [17]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6328 entries, 0 to 6327
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       6328 non-null   int64 
 1   group    6328 non-null   object
 2   viewed   6328 non-null   Int64 
 3   clicked  6328 non-null   Int64 
dtypes: Int64(2), int64(1), object(1)
memory usage: 210.2+ KB


In [18]:
df_4.clicked.value_counts()

0    4468
1    1860
Name: clicked, dtype: Int64

In [19]:
df_4['clicked_thru'] = np.where(df_4['clicked'] == 1, 1, 0)
df_4

,id,group,viewed,clicked,clicked_thru
0,182988,experiment,1,0,0
1,182994,control,1,1,1
2,183089,control,1,0,0
3,183136,experiment,1,0,0
4,183141,experiment,1,1,1
...,...,...,...,...,...
6323,937003,control,1,0,0
6324,937073,control,1,0,0
6325,937108,control,1,0,0
6326,937139,experiment,1,1,1


In [20]:
# df_4.loc[(df_4['viewed'] == 1) & (df_4['clicked'] == 1), 'clicked_thru_v2'] = 1

In [21]:
df_4.dtypes

id               int64
group           object
viewed           Int64
clicked          Int64
clicked_thru     int64
dtype: object

In [46]:
df_4.viewed.value_counts()

1    6328
Name: viewed, dtype: Int64

In [30]:
df_4.clicked_thru.value_counts()

0    4468
1    1860
Name: clicked_thru, dtype: int64

In [ ]:
# How many viewers also clicked? 
# Is there any overlap between the control and experiment groups? * 
    # If so, how do you plan to account for this in your experimental design?

In [32]:
# Are there any anomalies with the data; did anyone click who didn't view?
len(df_4.loc[(df_4['clicked'] == 1) & (df_4['viewed'] == 0)])

0

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [59]:
# group counts
control_count = len(df_4.loc[df_4['group'] == 'control'])
experiment_count = len(df_4.loc[df_4['group'] == 'experiment'])
print(control_count, experiment_count)

3332 2996


In [45]:
df_4.groupby('group')['viewed', 'clicked_thru'].sum()

/var/folders/wf/s7j2jn9j1ns035j4rx9g3rzr0000gn/T/ipykernel_13333/1692119224.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_4.groupby('group')['viewed', 'clicked_thru'].sum()


,viewed,clicked_thru
group,,
control,3332,932
experiment,2996,928


In [49]:
df_4.groupby('group')['clicked_thru'].mean().reset_index(name='avg_click_rate')

,group,avg_click_rate
0,control,0.279712
1,experiment,0.309746


In [53]:
import flatiron_stats as fs

In [56]:
control_clicks = df_4.loc[df_4['group'] == 'control'].clicked
experiment_clicks = df_4.loc[df_4['group'] == 'experiment'].clicked

In [57]:
fs.p_value_welch_ttest(control_clicks, experiment_clicks)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [60]:
#Your code here
control_rate = control_clicks.mean()
expexted_experiment_clicks = control_rate * experiment_count
expexted_experiment_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [62]:
#Your code here
n = experiment_count
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)
std

24.568547907005815

In [66]:
z_score = (experiment_clicks.sum() - expexted_experiment_clicks)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [67]:
#Your code here
import scipy.stats as stats
p_value = stats.norm.sf(z_score)
p_value

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: aye!

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.